In [ ]:
from dataclasses import dataclass
import numpy as np


# lng:  112 - 154:    (0 - 42)+112:       (range of 43 options, 6 bits)
# lat:  -10 - -44:    ((0 - 34)+10)*-1:   (range of 35 options, 6 bits)
# 8 degrees of precision requires 26 bits for a total of 32 bits
# -1 because we need to do maths on these numbers, and don't want overflow

PRECISION = 7
LAT_ADJUST = -10
LNG_ADJUST = 112


def to_lat(lat: float) -> np.uint32:
    return np.uint32(round((lat - LAT_ADJUST) * -(10**PRECISION)))


def from_lat(lat: np.uint32) -> float:
    return round(float(lat) / -(10**PRECISION) + LAT_ADJUST, PRECISION)


def to_lng(lng: float) -> np.uint32:
    return np.uint32(round((lng - LNG_ADJUST) * 10**PRECISION))


def from_lng(lng: np.uint32) -> float:
    return round(float(lng) / (10**PRECISION) + LNG_ADJUST, PRECISION)


@dataclass(frozen=True, slots=True)
class Geom:
    lat: np.uint32
    lng: np.uint32


@dataclass(frozen=True, slots=True)
class Polygon:
    points: list[Geom]


@dataclass(frozen=True, slots=True)
class BoundingBox:
    lng_min: np.uint32  # xmin
    lng_max: np.uint32  # xmax
    lat_min: np.uint32  # ymin
    lat_max: np.uint32  # ymax

def get_box_from_poly(poly: Polygon) -> BoundingBox:
    first_point = poly.points[0]
    lng_min = lng_max = first_point.lng
    lat_min = lat_max = first_point.lat
    for point in poly.points[1:]:
        lng_min = umin(lng_min, point.lng)
        lng_max = umax(lng_max, point.lng)
        lat_min = umin(lat_min, point.lat)
        lat_max = umax(lat_max, point.lat)
    return BoundingBox(lng_min, lng_max, lat_min, lat_max)


def calculate_volume(bb: BoundingBox) -> np.uint64:
    return np.uint64(bb.lat_max - bb.lat_min) * np.uint64(bb.lng_max - bb.lng_min)


def is_clockwise(poly: Polygon) -> bool:
    return bool(sum(((from_lng(poly.points[i].lng) - from_lng(poly.points[i - 1].lng)) * (from_lat(poly.points[i].lat) + from_lat(poly.points[i - 1].lat)) for i in range(len(poly.points)))) > 0)


def is_overlap(box_1: BoundingBox, box_2: BoundingBox) -> bool:
    return bool(box_1.lng_max > box_2.lng_min and box_1.lng_min < box_2.lng_max and box_1.lat_max > box_2.lat_min and box_1.lat_min < box_2.lat_max)


def fully_encompasses(box_1: BoundingBox, box_2: BoundingBox) -> bool:
    return bool(box_1.lat_min < box_2.lat_min and box_1.lat_max > box_2.lat_max and box_1.lng_min < box_2.lng_min and box_1.lng_max > box_2.lng_max)


def calculate_overlap(bb1: BoundingBox, bb2: BoundingBox) -> np.uint64:
    if is_overlap(bb1, bb2):
        return calculate_volume(BoundingBox(
            lng_min=umax(bb1.lng_min, bb2.lng_min),
            lng_max=umin(bb1.lng_max, bb2.lng_max),
            lat_min=umax(bb1.lat_min, bb2.lat_min),
            lat_max=umin(bb1.lat_max, bb2.lat_max),
        ))
    return np.uint64(0)


def join_boxes(bb1: BoundingBox, bb2: BoundingBox) -> BoundingBox:
    return BoundingBox(
        lng_min=umin(bb1.lng_min, bb2.lng_min),
        lng_max=umax(bb1.lng_max, bb2.lng_max),
        lat_min=umin(bb1.lat_min, bb2.lat_min),
        lat_max=umax(bb1.lat_max, bb2.lat_max),
    )


def umin(u1: np.uint32, u2: np.uint32) -> np.uint32:
    return u1 if u1 < u2 else u2


def umax(u1: np.uint32, u2: np.uint32) -> np.uint32:
    return u1 if u1 > u2 else u2


# Test

lat = -34.886284349000036
lng = 138.51091810299997
print(f"lat: {lat} -> {to_lat(lat)} -> {from_lat(to_lat(lat))}")
print(f"lng: {lng} -> {to_lng(lng)} -> {from_lng(to_lng(lng))}")


In [ ]:
from dataclasses import dataclass
from typing import Optional, TypeAlias
from enum import Enum


@dataclass(frozen=True, slots=True)
class BoxNode:
    bounding_box: BoundingBox
    left: "BoxNode | AABB"
    right: "BoxNode | AABB"


@dataclass(frozen=True, slots=True)
class AABB:
    bounding_box: BoundingBox
    id: str
    geometry: Polygon


AABBNode: TypeAlias = BoxNode | AABB


def add_box_to_tree(base_node: AABBNode, id: str, new_value: Polygon, new_box: BoundingBox) -> AABBNode:
    joined_box = join_boxes(base_node.bounding_box, new_box)

    if isinstance(base_node, AABB):
        left = base_node
        right = AABB(
            bounding_box=new_box,
            id = id,
            geometry = new_value
        )
        return BoxNode(bounding_box=joined_box, left = left, right = right)

    else:
        # merge the boxes
        left_join = join_boxes(base_node.left.bounding_box, new_box)
        right_join = join_boxes(base_node.right.bounding_box, new_box)

        # Find the merge volumes
        base_join_volume = calculate_volume(joined_box)
        left_join_volume = calculate_volume(left_join)
        right_join_volume = calculate_volume(right_join)

        # Find the overlap amounts
        base_join_overlap = calculate_overlap(base_node.bounding_box, new_box)
        left_join_overlap = calculate_overlap(base_node.right.bounding_box, left_join)
        right_join_overlap = calculate_overlap(base_node.left.bounding_box, right_join)

        # Calculate the costs
        base_volume_change = base_join_volume - calculate_volume(base_node.bounding_box)
        base_cost = base_join_volume + base_join_overlap
        left_cost = base_volume_change + (left_join_volume - calculate_volume(base_node.left.bounding_box)) + left_join_overlap
        right_cost = base_volume_change + (right_join_volume - calculate_volume(base_node.right.bounding_box)) + right_join_overlap

        if base_cost < left_cost and base_cost < right_cost:
            return BoxNode(
                bounding_box=joined_box,
                left = base_node,
                right = AABB(
                    bounding_box=new_box,
                    id=id,
                    geometry=new_value
                )
            )
        elif left_cost < right_cost:
            return BoxNode(
                bounding_box=joined_box,
                left = add_box_to_tree(base_node.left, id, new_value, new_box),
                right = base_node.right
            )
        else:
            return BoxNode(
                bounding_box=joined_box,
                left = base_node.left,
                right = add_box_to_tree(base_node.right, id, new_value, new_box)
            )


In [ ]:
from typing import Generator


def str_to_geom(coord_str: str) -> Geom:
    lng, lat = coord_str.split(" ", 1)
    return Geom(
        lat=to_lat(float(lat)),
        lng=to_lng(float(lng)),
    )

@dataclass(frozen=True, slots=True)
class Output:
    id: str
    polygons: list[Polygon]

@dataclass(frozen=True, slots=True)
class OutputPolygon:
    id: str
    polygon: Polygon

def extract_data_from_line(line: str) -> Generator[OutputPolygon]:
    elements = line.split("|", 2)
    id = elements[0][3:]
    polys = map(lambda polygon: Polygon(list(map(str_to_geom, polygon.removesuffix(")").split(", ")))), elements[2].removeprefix("POLYGON((").removesuffix(")").split(", ("))
    for poly in polys:
        yield OutputPolygon(id, poly)

def read_file_into_lines(file_name: str) -> Generator[OutputPolygon]:
    with open(file_name, "r") as r:
        for line in r.readlines():
            for poly in extract_data_from_line(line.strip()):
                yield poly


In [ ]:
# Efficiency be damned, it's getting late in the day and I can't be bothered doing geometric collision detection again

from shapely.geometry import Polygon as ShapePolygon


def to_shapely_polygon(poly: Polygon) -> ShapePolygon:
    return ShapePolygon(np.array([[point.lng, point.lat] for point in poly.points], dtype=np.uint32))


def do_polygons_intersects(poly1: Polygon, poly2: Polygon) -> bool:
    shape_poly1 = to_shapely_polygon(poly1)
    shape_poly2 = to_shapely_polygon(poly2)
    return shape_poly1.intersects(shape_poly2)

def get_intersects(output: BQOutputPolygon, tree: AABBNode) -> list[str]:
    polygon_bb = get_box_from_poly(output.polygon)
    bb_overlaps = is_overlap(tree.bounding_box, polygon_bb)
    if not bb_overlaps:
        return []
    elif isinstance(tree, GeoscapeAABB):
        if do_polygons_intersects(tree.geometry, output.polygon):
            return [tree.id]
    else:
        left_overlaps = get_intersects(output, tree.left) if tree.left is not None else []
        right_overlaps = get_intersects(output, tree.right) if tree.right is not None else []
        return left_overlaps + right_overlaps
    return []


In [ ]:
new_polys = read_file_into_lines("./data/property.csv")
first_poly = next(new_polys)

tree = GeoscapeAABB(
    bounding_box = get_box_from_poly(first_poly.polygon),
    id=first_poly.id,
    geometry=first_poly.polygon
)

for poly in new_polys:
    tree = add_box_to_tree(tree, poly.id, poly.polygon, get_box_from_poly(poly.polygon))


In [ ]:
# Let's visualise the bounding boxes as an image now

from PIL import Image

tree_img = Image.open(f'./visualisations/tree_visualisation.png')
mini_img = Image.open(f'./visualisations/tree_visualisation_mini.png')
size_differential = tree_img.size[1] / mini_img.size[1] / 3
mini_img = mini_img.resize((int(mini_img.size[0] * size_differential), int(mini_img.size[1] * size_differential)))

joined_image = Image.new('RGB', tree_img.size)
joined_image.paste(tree_img, (0, 0))
joined_image.paste(mini_img, (tree_img.size[0] - mini_img.size[0], 0))

display(joined_image.resize((joined_image.size[0] // 2, joined_image.size[1] // 2)))


In [ ]:
# Seriously though, let's visualise the bounding boxes as an image now

from PIL import Image

tree_img = Image.open(f'./visualisations/tree_visualisation_messy.png')

display(tree_img)

In [ ]:
# We can go one further with visualising the tree

from PIL import Image
import numpy as np

PATH_COLOUR = (np.uint8(255), np.uint8(255), np.uint8(255))
BRANCH_COLOUR = (np.uint8(255), np.uint8(255), np.uint8(255))
LEAF_COLOUR = (np.uint8(255), np.uint8(255), np.uint8(255))

class NodeType(Enum):
    BRANCH = 0
    LEAF = 1

@dataclass(frozen=True, slots=True)
class TreeNodeRepr:
    node_type: NodeType
    depth: int
    node_pos: int
    width: int
    height: int
    left: Optional['TreeNodeRepr']
    right: Optional['TreeNodeRepr']

def get_tree_node_repr(node: AABBNode, depth: int, offset: int) -> TreeNodeRepr:
    match node:
        case BoxNode(_, left, right):
            left_repr = get_tree_node_repr(left, depth + 1, offset)
            right_repr = get_tree_node_repr(right, depth + 1, offset + left_repr.width + 1)
            return TreeNodeRepr(NodeType.BRANCH, depth, offset + left_repr.width, left_repr.width + 1 + right_repr.width, max(left_repr.height, right_repr.height) + 1, left_repr, right_repr)
        case GeoscapeAABB(_, _, _, _):
            return TreeNodeRepr(NodeType.LEAF, depth, offset + 1, 3, 1, None, None)

def paint_tree_repr_to_image(tree_repr: TreeNodeRepr, pixel_map: np.ndarray, parent_pos: Optional[tuple[int, int]]) -> np.ndarray:
    pos = (tree_repr.node_pos * 3, tree_repr.depth * 3 * 2)
    if tree_repr.left is not None and tree_repr.right is not None:
        pixel_map[pos[1] + 3, pos[0] + 1] = PATH_COLOUR
        pixel_map[pos[1] + 4, pos[0] + 1] = PATH_COLOUR
        pixel_map = paint_tree_repr_to_image(tree_repr.left, pixel_map, pos)
        pixel_map = paint_tree_repr_to_image(tree_repr.right, pixel_map, pos)
    if parent_pos is not None:
        # Only not the case when it is the base node
        pixel_map[pos[1] - 2, pos[0] + 1] = PATH_COLOUR
        pixel_map[pos[1] - 1, pos[0] + 1] = PATH_COLOUR
        for x_coord in range(parent_pos[0] + 1, pos[0] + 1, (pos[0] - parent_pos[0])//abs(pos[0] - parent_pos[0])):
            pixel_map[pos[1] - 2, x_coord] = PATH_COLOUR
    match tree_repr.node_type:
        case NodeType.BRANCH:
            # Draw a box
            pixel_map[pos[1], pos[0]] = BRANCH_COLOUR
            pixel_map[pos[1], pos[0] + 1] = BRANCH_COLOUR
            pixel_map[pos[1], pos[0] + 2] = BRANCH_COLOUR
            pixel_map[pos[1] + 1, pos[0] + 2] = BRANCH_COLOUR
            pixel_map[pos[1] + 2, pos[0] + 2] = BRANCH_COLOUR
            pixel_map[pos[1] + 2, pos[0] + 1] = BRANCH_COLOUR
            pixel_map[pos[1] + 2, pos[0]] = BRANCH_COLOUR
            pixel_map[pos[1] + 1, pos[0]] = BRANCH_COLOUR
        case NodeType.LEAF:
            # Draw a square
            pixel_map[pos[1], pos[0]] = LEAF_COLOUR
            pixel_map[pos[1], pos[0] + 1] = LEAF_COLOUR
            pixel_map[pos[1], pos[0] + 2] = LEAF_COLOUR
            pixel_map[pos[1] + 1, pos[0]] = LEAF_COLOUR
            pixel_map[pos[1] + 1, pos[0] + 1] = LEAF_COLOUR
            pixel_map[pos[1] + 1, pos[0] + 2] = LEAF_COLOUR
            pixel_map[pos[1] + 2, pos[0]] = LEAF_COLOUR
            pixel_map[pos[1] + 2, pos[0] + 1] = LEAF_COLOUR
            pixel_map[pos[1] + 2, pos[0] + 2] = LEAF_COLOUR
    return pixel_map

tree_repr = get_tree_node_repr(tree, 0, 0)

pixel_map = paint_tree_repr_to_image(tree_repr, np.ndarray(shape=(tree_repr.height * 2 * 3, tree_repr.width * 3, 3), dtype=np.uint8), None)
print(pixel_map.shape)

Image.fromarray(pixel_map).save("./visualisations/tree.png")

In [ ]:
cad_polys = read_file_into_lines("./data/cadastre.csv")
with open("./data/intersects.txt", "w") as w:
    w.write("")
with open("./data/intersects.txt", "a") as w:
    for poly in cad_polys:
        intersects = get_intersects(poly, tree)
        if len(intersects) > 0:
            w.write(f"{poly.id} intersects with {intersects}\n")
